[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/gpt4-retrieval-augmentation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/generation/gpt4-retrieval-augmentation.ipynb)

In [1]:
# !pip install -qU bs4 tiktoken openai langchain pinecone-client[grpc]

In [1]:
import requests

res = requests.get("https://langchain.readthedocs.io/en/latest/")
res

<Response [200]>

In [2]:
from bs4 import BeautifulSoup
import urllib.parse
import html
import re

domain = "https://langchain.readthedocs.io/"
domain_full = domain + "en/latest/"

soup = BeautifulSoup(res.text, 'html.parser')

# Find all links to local pages on the website
local_links = []
for link in soup.find_all('a', href=True):
    href = link['href']
    if href.startswith(domain) or href.startswith('./') \
        or href.startswith('/') or href.startswith('modules') \
        or href.startswith('use_cases'):
        local_links.append(urllib.parse.urljoin(domain_full, href))

# Find the main content using CSS selectors
main_content = soup.select('body main')[0]

# Extract the HTML code of the main content
main_content_html = str(main_content)

# Extract the plaintext of the main content
main_content_text = main_content.get_text()

# Remove all HTML tags
main_content_text = re.sub(r'<[^>]+>', '', main_content_text)

# Remove extra white space
main_content_text = ' '.join(main_content_text.split())

# Replace HTML entities with their corresponding characters
main_content_text = html.unescape(main_content_text)

print(main_content_text)

.rst .pdf Welcome to LangChain Contents Getting Started Modules Use Cases Reference Docs LangChain Ecosystem Additional Resources Welcome to LangChain# Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. But using these LLMs in isolation is often not enough to create a truly powerful app - the real power comes when you are able to combine them with other sources of computation or knowledge. This library is aimed at assisting in the development of those types of applications. Common examples of these types of applications include: ❓ Question Answering over specific documents Documentation End-to-end Example: Question Answering over Notion Database 💬 Chatbots Documentation End-to-end Example: Chat-LangChain 🤖 Agents Documentation End-to-end Example: GPT+WolframAlpha Getting Started# Checkout the below guide for a walkthrough of how to get started using LangChain to create an Language Model app

In [3]:
def scrape(url: str):
    res = requests.get(url)
    if res.status_code != 200:
        print(f"{res.status_code} for '{url}'")
        return None
    soup = BeautifulSoup(res.text, 'html.parser')

    # Find all links to local pages on the website
    local_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith(domain) or href.startswith('./') \
            or href.startswith('/') or href.startswith('modules') \
            or href.startswith('use_cases'):
            local_links.append(urllib.parse.urljoin(domain_full, href))

    # Find the main content using CSS selectors
    main_content = soup.select('body main')[0]

    # Extract the HTML code of the main content
    main_content_html = str(main_content)

    # Extract the plaintext of the main content
    main_content_text = main_content.get_text()

    # Remove all HTML tags
    main_content_text = re.sub(r'<[^>]+>', '', main_content_text)

    # Remove extra white space
    main_content_text = ' '.join(main_content_text.split())

    # Replace HTML entities with their corresponding characters
    main_content_text = html.unescape(main_content_text)

    # return as json
    return {
        "url": url,
        "text": main_content_text
    }, local_links

In [49]:
links = ["https://langchain.readthedocs.io/en/latest/"]
scraped = set()
data = []

while True:
    if len(links) == 0:
        print("Complete")
        break
    url = links[0]
    print(url)
    res = scrape(url)
    scraped.add(url)
    if res is not None:
        page_content, local_links = res
        data.append(page_content)
        # add new links to links list
        links.extend(local_links)
        # remove duplicates
        links = list(set(links))
    # remove links already scraped
    links = [link for link in links if link not in scraped]

    if len(scraped) >= 100:
        break

https://langchain.readthedocs.io/en/latest/
https://langchain.readthedocs.io/en/latest/modules/document_loaders/examples/s3_file.html
https://langchain.readthedocs.io/en/latest/modules/indexes/vectorstore_examples/elasticsearch.html
https://langchain.readthedocs.io/en/latest/modules/agents/examples/human_tools.html
https://langchain.readthedocs.io/en/latest/modules/document_loaders/examples/url.html
https://langchain.readthedocs.io/en/latest/use_cases/evaluation/qa_benchmarking_pg.html
https://langchain.readthedocs.io/en/latest/modules/document_loaders/examples/gutenberg.html
https://langchain.readthedocs.io/en/latest/modules/prompts/how_to_guides.html
https://langchain.readthedocs.io/en/latest/examples/custom_prompt_template.html
404 for 'https://langchain.readthedocs.io/en/latest/examples/custom_prompt_template.html'
https://langchain.readthedocs.io/en/latest/modules/memory/getting_started.html
https://langchain.readthedocs.io/en/latest/modules/memory/types/buffer_window.html
https:/

In [50]:
data[3]

{'url': 'https://langchain.readthedocs.io/en/latest/modules/agents/examples/human_tools.html',
 'text': '.ipynb .pdf Human as a tool Human as a tool# Human are AGI so they can certainly be used as a tool to help out AI agent when it is confused. import sys from langchain.chat_models import ChatOpenAI from langchain.llms import OpenAI from langchain.agents import load_tools, initialize_agent llm = ChatOpenAI(temperature=0.0) math_llm = OpenAI(temperature=0.0) tools = load_tools( ["human", "llm-math"], llm=math_llm, ) agent_chain = initialize_agent( tools, llm, agent="zero-shot-react-description", verbose=True, ) In the above code you can see the tool takes input directly from command line. You can customize prompt_func and input_func according to your need. agent_chain.run("What is Eric Zhu\'s birthday?") # Answer with "last week" > Entering new AgentExecutor chain... I don\'t know Eric Zhu, so I should ask a human for guidance. Action: Human Action Input: "Do you know when Eric Zhu\'s 

It's pretty ugly but it's good enough for now. Let's see how we can process all of these. We will chunk everything into ~1000 token chunks, we can do this easily with `langchain` and `tiktoken`:

In [51]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    
    return len(tokens)

In [52]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

Process the `data` into more chunks using this approach.

In [53]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record['url']
    } for i in range(len(texts))])

  0%|          | 0/81 [00:00<?, ?it/s]

In [54]:
len(chunks)

369

In [55]:
chunks[:3]

[{'id': '353f24c1-f1f0-4aac-b0fa-b6e1f2c5b8cf',
  'text': '.rst .pdf Welcome to LangChain Contents Getting Started Modules Use Cases Reference Docs LangChain Ecosystem Additional Resources Welcome to LangChain# Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. But using these LLMs in isolation is often not enough to create a truly powerful app - the real power comes when you are able to combine them with other sources of computation or knowledge. This library is aimed at assisting in the development of those types of applications. Common examples of these types of applications include: ❓ Question Answering over specific documents Documentation End-to-end Example: Question Answering over Notion Database 💬 Chatbots Documentation End-to-end Example: Chat-LangChain 🤖 Agents Documentation End-to-end Example: GPT+WolframAlpha Getting Started# Checkout the below guide for a walkthrough of how to 

Our chunks are ready so now we move onto embedding and indexing everything.

## Initialize Embedding Model

We use `text-embedding-ada-002` as the embedding model. We can embed text like so:

In [56]:
import openai
from getpass import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

# initialize openai API key
openai.api_key = os.environ["OPENAI_API_KEY"]  #platform.openai.com

embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ],
    engine=embed_model
)

In the response `res` we will find a JSON-like object containing our new embeddings within the `'data'` field.

In [57]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Inside `'data'` we will find two records, one for each of the two sentences we just embedded. Each vector embedding contains `1536` dimensions (the output dimensionality of the `text-embedding-ada-002` model.

In [58]:
len(res['data'])

2

In [59]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

We will apply this same embedding logic to the langchain docs dataset we've just scraped. But before doing so we must create a place to store the embeddings.

## Initializing the Index

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [60]:
import pinecone

# os.environ["PINECONE_API_KEY"] = getpass("Pinecone API Key: ")
# os.environ["PINECONE_ENVIRONMENT"] = input("Pinecone Environment: ")

In [61]:
index_name = 'gpt-4-langchain-docs'

# initialize connection to pinecone
pinecone.init(
    api_key=os.environ["PINECONE_API_KEY"],  # app.pinecone.io (console)
    environment=os.environ["PINECONE_ENVIRONMENT"]  # next to API key in console
)

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='dotproduct'
    )
# connect to index
index = pinecone.GRPCIndex(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it with OpenAI `text-embedding-ada-002` built embeddings like so:

In [62]:
len(chunks)

369

In [63]:
min(len(chunks), 100)

100

In [64]:
list(range(0, len(chunks), batch_size))

[0, 100, 200, 300]

In [65]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):

    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]

    # get ids
    ids_batch = [x['id'] for x in meta_batch]

    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]

    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/4 [00:00<?, ?it/s]

Now we've added all of our langchain docs to the index. With that we can move on to retrieval and then answer generation using GPT-4.

## Retrieval

To search through our documents we first need to create a query vector `xq`. Using `xq` we will retrieve the most relevant chunks from the LangChain docs, like so:

In [66]:
query = "how do I use the LLMChain in LangChain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)

In [67]:
res

{'matches': [{'id': '7c2c12b3-0b54-4300-b357-734246eefbc5',
              'metadata': {'chunk': 1.0,
                           'text': 'chain first uses a LLM to construct the '
                                   'url to hit, then makes that request with '
                                   'the Requests wrapper, and finally runs '
                                   'that result through the language model '
                                   'again in order to product a natural '
                                   'language response. Example Notebook '
                                   'LLMBash Chain Links Used: BashProcess, '
                                   'LLMChain Notes: This chain takes user '
                                   'input (a question), uses an LLM chain to '
                                   'convert it to a bash command to run in the '
                                   'terminal, and then returns that as the '
                                   'result. Exampl

With retrieval complete, we move on to feeding these into GPT-4 to produce answers.

## Retrieval Augmented Generation

GPT-4 is currently accessed via the `ChatCompletions` endpoint of OpenAI. To add the information we retrieved into the model, we need to pass it into our user prompts *alongside* our original query. We can do that like so:

In [68]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts) + "\n\n-----\n\n" + query

In [69]:
print(augmented_query)

chain first uses a LLM to construct the url to hit, then makes that request with the Requests wrapper, and finally runs that result through the language model again in order to product a natural language response. Example Notebook LLMBash Chain Links Used: BashProcess, LLMChain Notes: This chain takes user input (a question), uses an LLM chain to convert it to a bash command to run in the terminal, and then returns that as the result. Example Notebook LLMChecker Chain Links Used: LLMChain Notes: This chain takes user input (a question), uses an LLM chain to answer that question, and then uses other LLMChains to self-check that answer. Example Notebook LLMRequests Chain Links Used: Requests, LLMChain Notes: This chain takes a URL and other inputs, uses Requests to get the data at that URL, and then passes that along with the other inputs into an LLMChain to generate a response. The example included shows how to ask a question to Google - it firsts constructs a Google url, then fetches t

Now we ask the question:

In [70]:
# system message to 'prime' the model
primer = f"""
You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know".
""".strip()

res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}
    ]
)

To display this response nicely, we will display it in markdown.

In [71]:
from IPython.display import Markdown

display(Markdown(res['choices'][0]['message']['content']))

To use the LLMChain in LangChain, you need to follow these steps:

1. Import the required libraries and classes:

```python
from langchain import PromptTemplate, LLMChain
from langchain.llms import AI21  # Or any other LLM you want to use
```

2. Define a PromptTemplate:

```python
template = """Question: {question} Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
```

3. Create an instance of the LLM you want to use (e.g., AI21):

```python
llm = AI21()
```

4. Create an LLMChain with the defined prompt and LLM instance:

```python
llm_chain = LLMChain(prompt=prompt, llm=llm)
```

5. Run the LLMChain with a question as input:

```python
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
response = llm_chain.run(question)
```

In this example, the LLMChain combines the PromptTemplate, LLM, and input question to generate a response using the AI21 large language model. You can customize the template and use different LLMs based on your requirements.

Let's compare this to a non-augmented query...

In [72]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": primer},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

I don't know.

If we drop the `"I don't know"` part of the `primer`?

In [73]:
res = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},
        {"role": "user", "content": query}
    ]
)
display(Markdown(res['choices'][0]['message']['content']))

LangChain is not a well-known technology, and there isn't enough information available for a specific term like "LLMChain." Therefore, it's not possible for me to provide you with accurate guidance on how to use the LLMChain in LangChain.

However, if you're referring to a custom library or framework, your best course of action would be to consult the developer who created the technology or refer to the documentation provided for that specific library or framework.